[Dokumentacja mysql.connector](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)

Funkcja connect() tworzy połączenie z MySQL server i zwraca obiekt MySQLConnection:
```python
cnx = mysql.connector.connect(**config_dict)
```

Metoda połączenia cursor() zwraca obiekt MySQLCursor, który umożliwia manipulowanie bazą:
```python
cursor = cnx.cursor()

cursor.execute('sql_statement'[, data])
cursor.executemany('sql_statement', seq_of_data)
# w wyniku zapytania cursor może zostać użyty jako iterator lub można pobrać wyszukane wiersze
row_list = cursor.fetchall()

iterator = cursor.execute('many_sql_statement', multi=True)
```

Po dodaniu (INSERT INTO), zaktualizowaniu (UPDATE) lub usunięciu (DELETE FROM) rekordów, zmiany należy jeszcze skomitować:
```
cnx.commit()
```

Połączenie należy zakończyć:
```
cursor.close()
cnx.close()
```

In [1]:
import mysql.connector
import random

### Nawiązanie połączenia z bazą danych

Utworzenie bazy danych:
```mysql
CREATE DATABASE database_name;
```

In [2]:
db_name = 'cardb'

In [3]:
# słownik złożony z argumentów kluczowych funkcji connect()
config = {
  'user': 'root',
  'password': 'admin',
  'host': '127.0.0.1',
#   'database': db_name,
}

In [4]:
# połączenie się z serwerem (lub konkretną bazą na tym serwerze, jeżeli przekazany jest też argument database)
cnx = mysql.connector.connect(**config)
cursor = cnx.cursor()

In [5]:
# połączenie się z konkretną bazą
# jeżeli baza nie istnieje, zostaje utworzona
try:
    cnx.database = db_name
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_ACCESS_DENIED_ERROR:
    print('Something is wrong with your user name or password')
  elif err.errno == mysql.connector.errorcode.ER_BAD_DB_ERROR:
#     print('Database {} does not exist'.format(db_name))
    cursor.execute('CREATE DATABASE {}'.format(db_name))
    cnx.database = db_name
  else:
    print(err)

### Utworzenie tabel

Utworzenie nowej tabeli:
```mysql
CREATE TABLE table_name (
    column1 datatype constraint,
    column2 datatype constraint,
    column3 datatype constraint,
    ...
    PRIMARY KEY (column_name)
    /*CONSTRAINT contraint_name PRIMARY KEY (column_name(s))*/
    FOREIGN KEY (column_name) REFERENCES table2_name(column_name) 
    /*CONSTRAINT contraint_name FOREIGN KEY (column_name(s)) REFERENCES table2_name(column_name)*/
);
```

Utworzenie nowej tabeli na podstawie już istniejących:
```mysql
CREATE TABLE table_name AS
zapytanie;
```

In [6]:
table_dict = {}

In [7]:
table_dict['Car'] = '''
    CREATE TABLE `Car` (
    `Id` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`Id`))
    '''

table_dict['Part'] = '''
    CREATE TABLE `Part` (
    `Id` INT NOT NULL,
    `IsFunctional` TINYINT(1) NOT NULL,
    `CarId` INT,
    PRIMARY KEY (`Id`),
    FOREIGN KEY (`CarId`) REFERENCES `Car`(`Id`))
    '''

table_dict['Wheel'] = '''
    CREATE TABLE `Wheel` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

table_dict['Engine'] = '''
    CREATE TABLE `Engine` (
    `PartId` INT NOT NULL,
    `Model` VARCHAR(45) NOT NULL,
    PRIMARY KEY (`PartId`),
    FOREIGN KEY (`PartId`) REFERENCES `Part`(`Id`))
    '''

In [8]:
# utworzenie tabel
# jeżeli już istnieją, pojawi się komunikat
for table_name, create_table_stmt in table_dict.items():
    try:
        cursor.execute(create_table_stmt)
    except mysql.connector.Error as err:
      if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
        print('Table {} already exists'.format(table_name))
      else:
        print(err)

### Dodanie wierszy

Wstawienie nowych rekordów do tabeli:
```mysql
INSERT INTO table_name (column1, column2, column3, ...)
VALUES (value1, value2, value3, ...), (value1, value2, value3, ...), ...;
```

In [9]:
add_car_stmt = '''
    INSERT INTO Car (Id, Model)
    VALUES (%s, %s)
    '''

add_part_stmt = '''
    INSERT INTO Part (Id, IsFunctional, CarId)
    VALUES (%s, %s, %s)
    '''

add_wheel_stmt = '''
    INSERT INTO Wheel (PartId, Model)
    VALUES (%s, %s)
    '''

add_engine_stmt = '''
    INSERT INTO Engine (PartId, Model)
    VALUES (%s, %s)
    '''

In [10]:
cursor.execute('SELECT Id FROM Car')
try:
    last_car_id = cursor.fetchall()[-1][0]
except IndexError:
    last_car_id = 0

car_total = 20
car_id_list = list(range(last_car_id + 1, last_car_id + 1 + car_total))
car_model_list = ['Renault', 'Audi', 'BMW', 'Mazda', 'Fiat']
car_list = [(car_id, random.choice(car_model_list)) for car_id in car_id_list]

cursor.execute('SELECT Id FROM Part')
try:
    last_part_id = cursor.fetchall()[-1][0]
except IndexError:
    last_part_id = 0

part_total = 100
part_id_list = list(range(last_part_id + 1, last_part_id + 1 + part_total))
part_list = [(part_id, random.randint(0, 1), random.choice(car_id_list)) for part_id in part_id_list]

wheel_total = int(part_total * 0.7)
wheel_model_list = ['W1', 'W2', 'W3', 'W4', 'W5']
wheel_part_id_list = random.sample(part_id_list, wheel_total)
wheel_list = [(part_id, random.choice(wheel_model_list)) for part_id in wheel_part_id_list]

engine_total = part_total - wheel_total
engine_model_list = ['A157', 'B458', 'A86', 'A123', 'C18']
engine_part_id_list = list(set(part_id_list) - set(wheel_part_id_list))
engine_list = [(part_id, random.choice(engine_model_list)) for part_id in engine_part_id_list]

In [11]:
cursor.executemany(add_car_stmt, car_list)

In [12]:
cursor.executemany(add_part_stmt, part_list)

In [13]:
cursor.executemany(add_wheel_stmt, wheel_list)

In [14]:
cursor.executemany(add_engine_stmt, engine_list)

In [15]:
cnx.commit()

### Zaktualizowanie wierszy
Zmodyfikowanie istniejących rekordów w tabeli:
```mysql
UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
```

In [16]:
set_free_part_stmt = '''
    UPDATE Part
    SET CarId = NULL
    WHERE Id IN {}
    '''

set_broken_part_stmt = '''
    UPDATE Part
    SET IsFunctional = 0
    WHERE Id = %s
    '''

set_fixed_part_stmt = '''
    UPDATE Part
    SET IsFunctional = 1
    WHERE Id = %s
    '''

In [17]:
free_part_total = 15
free_part_id_list = random.sample(part_id_list, free_part_total)
set_free_part_full_stmt = set_free_part_stmt.format(str(('%s',) * free_part_total))

In [18]:
cursor.execute(set_free_part_full_stmt, free_part_id_list)

In [19]:
fixed_part_id_list = [(part_id,) for part_id in random.sample(part_id_list, 10)]

In [20]:
cursor.executemany(set_fixed_part_stmt, fixed_part_id_list)

In [21]:
cnx.commit()

### Utworzenie widoku

Widok jest wirtualną tabelą opartą na wyniku zapytania. Można z niego korzystać tak, jak z rzeczywistych tabel:
```mysql
CREATE VIEW view_name AS
zapytanie;
```

In [22]:
# stwórz widok sprawnych samochodów
# samochód jest sprawny, jeżeli wszystkie jego części są sprawne
create_view_functional_car_stmt = '''
    CREATE VIEW FunctionalCar AS
    SELECT * FROM Car 
    WHERE Car.Id IN (SELECT CarId FROM Part
                     GROUP BY CarId
                     HAVING COUNT(Part.IsFunctional) = SUM(Part.IsFunctional))
    '''
# inny sposób:
#     SELECT Car.Id, Model
#     FROM Car INNER JOIN Part ON Car.Id = Part.CarId
#     GROUP BY Car.Id
#     HAVING COUNT(IsFunctional) = SUM(IsFunctional)

In [23]:
try:
    cursor.execute(create_view_functional_car_stmt)
except mysql.connector.Error as err:
  if err.errno == mysql.connector.errorcode.ER_TABLE_EXISTS_ERROR:
    print('View already exists')
  else:
    print(err)

### Zapytania

Schemat zapytania:
```mysql
SELECT column_name(s)
FROM table_name
WHERE condition 
GROUP BY column_name(s)
HAVING condition
ORDER BY column_name(s)
LIMIT no_of_rows;
```

Połączenie tabel względem pasujących do siebie wierszy:
```mysql
SELECT column_name(s)
FROM table1
INNER JOIN table2 /*INNER/FULL OUTER/LEFT/RIGHT JOIN*/
ON table1.column_name = table2.column_name;
```

Połączenie dwóch wyników zapytań - muszą być te same kolumny:
```mysql
SELECT column_name(s) FROM table1
UNION /*UNION ALL pozwala na duplikaty*/
SELECT column_name(s) FROM table2;
```

In [24]:
def print_res_of_query(cursor):
    frame_str = '{:<12} ' * len(cursor.column_names)
    print(frame_str.format(*cursor.column_names))
    for row in cursor:
        print(frame_str.format(*row))    

* #### Znajdź sprawne samochody
Samochód jest sprawny, jeżeli wszystkie jego części są sprawne

In [25]:
get_functional_car_query = '''
    SELECT * FROM FunctionalCar
    '''

In [26]:
cursor.execute(get_functional_car_query)
print_res_of_query(cursor)

Id           Model        
3            Mazda        
5            Fiat         
6            Fiat         


* #### Znajdź liczbę sprawnych i niesprawnych samochodów danej marki

In [27]:
get_funct_and_nonfunct_model_car_query = '''
    SELECT Car.Model, COUNT(FunctionalCar.Id) AS FunctCar, COUNT(Car.Id) - COUNT(FunctionalCar.Id) AS NonfunctCar
    FROM Car LEFT JOIN FunctionalCar ON FunctionalCar.Id = Car.Id
    GROUP BY Model
    '''

In [28]:
cursor.execute(get_funct_and_nonfunct_model_car_query)
print_res_of_query(cursor)

Model        FunctCar     NonfunctCar  
Audi         0            5            
BMW          0            3            
Fiat         2            4            
Mazda        1            4            
Renault      0            1            


* #### Znajdź liczbę sprawnych i niesprawnych części dla konkretnych samochodów
Zapisz je na dwa sposoby - w dwóch oddzielnych kolumnach i wierszach

In [29]:
# oddzielne kolumny
get_car_funct_and_nonfunct_part_query = '''
    SELECT Car.Id, Model, SUM(IsFunctional) AS FunctPart, COUNT(IsFunctional) - SUM(IsFunctional) AS NonfunctPart
    FROM Car INNER JOIN Part ON Car.Id = Part.CarId
    GROUP BY Car.Id
    LIMIT 10
    '''

In [30]:
cursor.execute(get_car_funct_and_nonfunct_part_query)
print_res_of_query(cursor)

Id           Model        FunctPart    NonfunctPart 
1            Fiat         1            1            
2            Fiat         3            1            
3            Mazda        2            0            
4            Audi         2            1            
5            Fiat         1            0            
6            Fiat         2            0            
7            Audi         2            2            
8            BMW          2            4            
9            Fiat         2            2            
10           Renault      2            6            


In [31]:
# oddzielne wiersze
get_car_funct_and_nonfunct_part_query = '''
    SELECT Car.Id, Model, IsFunctional, COUNT(IsFunctional) AS NoOfPart
    FROM Car INNER JOIN Part ON Car.Id = Part.CarId
    GROUP BY Car.Id, IsFunctional
    LIMIT 10
    '''

In [32]:
cursor.execute(get_car_funct_and_nonfunct_part_query)
print_res_of_query(cursor)

Id           Model        IsFunctional NoOfPart     
1            Fiat         0            1            
1            Fiat         1            1            
2            Fiat         0            1            
2            Fiat         1            3            
3            Mazda        1            2            
4            Audi         0            1            
4            Audi         1            2            
5            Fiat         1            1            
6            Fiat         1            2            
7            Audi         0            2            


* #### Znajdź wszystkie luźne części
Luźne części to te, które nie są związane z żadnym samochodem

In [33]:
get_free_part_query = '''
    SELECT Part.Id AS Id, 'Wheel' AS Type, Model, IsFunctional
    FROM Part INNER JOIN Wheel ON Part.Id = Wheel.PartId
    WHERE CarId IS NULL
    UNION
    SELECT Part.Id AS Id, 'Engine' AS Type, Model, IsFunctional
    FROM Part INNER JOIN Engine ON Part.Id = Engine.PartId
    WHERE CarId IS NULL
    ORDER BY Id
    LIMIT 10
    '''

In [34]:
cursor.execute(get_free_part_query)
print_res_of_query(cursor)

Id           Type         Model        IsFunctional 
5            Engine       C18          0            
8            Wheel        W4           0            
11           Wheel        W5           0            
14           Wheel        W5           0            
20           Wheel        W4           0            
39           Wheel        W2           0            
55           Engine       A157         0            
59           Engine       A157         0            
60           Engine       B458         1            
76           Wheel        W2           0            


* #### Znajdź samochody bez części

In [35]:
get_car_no_part_query = '''
    SELECT * FROM Car
    WHERE Id NOT IN (SELECT DISTINCT CarId FROM Part WHERE CarId IS NOT NULL)
    LIMIT 10
    '''

In [36]:
cursor.execute(get_car_no_part_query)
print_res_of_query(cursor)

Id           Model        


* #### Znajdź wszystkie części związane z samochodami

In [37]:
get_car_part_query = '''
    SELECT Car.Id AS CarId, Car.Model AS Car, Type AS Part, FullPart.Model AS PartModel, IsFunctional
    FROM Car INNER JOIN (
        SELECT *, 'Wheel' AS Type
        FROM Part INNER JOIN Wheel ON Part.Id = Wheel.PartId
        UNION
        SELECT *, 'Engine' AS Type
        FROM Part INNER JOIN Engine ON Part.Id = Engine.PartId) AS FullPart 
    ON FullPart.CarId = Car.Id
    ORDER BY CarId
    LIMIT 20
    '''

In [38]:
cursor.execute(get_car_part_query)
print_res_of_query(cursor)

CarId        Car          Part         PartModel    IsFunctional 
1            Fiat         Wheel        W4           1            
1            Fiat         Wheel        W3           0            
2            Fiat         Engine       C18          1            
2            Fiat         Engine       C18          1            
2            Fiat         Wheel        W5           1            
2            Fiat         Engine       A123         0            
3            Mazda        Wheel        W2           1            
3            Mazda        Engine       C18          1            
4            Audi         Wheel        W4           0            
4            Audi         Wheel        W5           1            
4            Audi         Wheel        W1           1            
5            Fiat         Engine       A123         1            
6            Fiat         Wheel        W1           1            
6            Fiat         Wheel        W5           1            
7         

* #### Znajdź dostępne modele i ich liczbę

In [39]:
get_model_query = '''
    SELECT Model, COUNT(*) AS Count FROM {}
    GROUP BY Model
    '''

table_name_list = ['Car', 'Wheel', 'Engine']

In [40]:
result_iter = cursor.execute('; '.join(map(get_model_query.format, table_name_list)), multi=True)
frame_str = '{:%s}: {}' % (max(map(len, table_name_list)) + 1)
for table_name, result in zip(table_name_list, result_iter):
    print(frame_str.format(table_name, result.fetchall()))

Car    : [('Audi', 5), ('BMW', 3), ('Fiat', 6), ('Mazda', 5), ('Renault', 1)]
Wheel  : [('W1', 17), ('W2', 14), ('W3', 8), ('W4', 16), ('W5', 15)]
Engine : [('A123', 9), ('A157', 6), ('A86', 2), ('B458', 2), ('C18', 11)]


### Usunięcie wierszy

```mysql
DELETE FROM table_name
WHERE condition;
```

### Usunięcie tabeli

```mysql
DROP TABLE table_name;
```

### Usunięcie bazy danych

```mysql
DROP DATABASE database_name;
```

In [41]:
# try:
#     cursor.execute('DROP DATABASE {}'.format(db_name))
# except mysql.connector.Error as err:
#     print(err)

### Zakończenie połączenia

In [42]:
cursor.close()
cnx.close()